# Find Similar Players

### Import Libraries

In [140]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import numpy as np

### Reading Datasets

In [141]:
players_df = pd.read_csv('players_df.csv')
gk_df = pd.read_csv('goalkeepers_df.csv')
eafc_pl_df = pd.read_csv('players_fifa_df.csv')
eafc_gk_df = pd.read_csv('goalkeepers_fifa_df.csv')

### Dropping Columns not to consider

In [143]:
eafc_pl_df = eafc_pl_df.drop(['Foot', 'Height', 'Weight', 'Nationality',
       'Expiration', 'Non-PenaltyGoals', 'npxG:Non-PenaltyxG', 'ShotsTotal',
       'Assists', 'xAG:Exp.AssistedGoals', 'npxG+xAG', 'Shot-CreatingActions',
       'PassesAttempted', 'PassCompletion%', 'ProgressivePasses',
       'ProgressiveCarries', 'SuccessfulTake-Ons', 'Touches(AttPen)',
       'ProgressivePassesRec', 'Tackles', 'Interceptions_x', 'Blocks',
       'Clearances', 'AerialsWon','Birth', 'Role'], axis=1)
eafc_pl_df = eafc_pl_df.dropna()
eafc_gk_df = eafc_gk_df.drop(['Foot', 'Height', 'Weight', 'Nationality', 'Wage',
       'Expiration', 'PSxG-GA', 'GoalsAgainst', 'SavePercentage', 'PSxG/SoT',
       'Save%(PenaltyKicks)', 'CleanSheetPercentage', 'Touches', 'Launch%',
       'GoalKicks', 'Avg.LengthofGoalKicks', 'CrossesStopped%',
       'Def.ActionsOutsidePen.Area', 'Avg.DistanceofDef.Actions', 'Role',
       'Birth'], axis=1)
eafc_gk_df = eafc_gk_df.dropna()

In [144]:
def find_similar_players(df: pd.DataFrame, player_index=None, player_name=None, top_n=5) -> pd.DataFrame:
    '''
    Finds the top_n similar players in the dataframe given to the player specified using cosine similarity
    
    Parameters
    ----------
    df : dict
        Dictionary containing players to be compared
    player_index : int
        Index in the dataframe of the player to get similar players of
    player_name : dict
        Name of the player to get similar players of
    top_n : int
        Number of most similar players to be returned
        
    Returns
    ----------
    df: dictionary
        Dictionary having top_n similar players in the dataframe
    '''
    if player_index is None and player_name is None:
        return None
    if player_name is not None:
        player_index = df.index[df['Player'] == player_name]
    df_copy = df.drop(['Age', 'Wage', 'Value'], axis=1)
    df_copy = df_copy.select_dtypes(include=['float64'])
    features = df_copy.columns
    scaler = StandardScaler()
    players_scaled = scaler.fit_transform(df_copy)
    players_scaled_df = pd.DataFrame(players_scaled, columns=features, index=df_copy.index)
    player_features = players_scaled_df.iloc[player_index].values
    distances = np.sqrt(np.sum((players_scaled_df - player_features) ** 2, axis=1))
    similar_indices = np.argsort(distances)[1:top_n+1]    
    return df.iloc[similar_indices]

In [145]:
df_fin = find_similar_players(players_df, player_name = 'Kevin De Bruyne')
df_fin[['Player', 'Club', 'League', 'Age', 'Position', 'Value', 'Age']]


,Player,Club,League,Age,Position,Value,Age
432,Raphinha,Barcelona,Spanish La Liga,27,Right Winger,50.0,27
19,Mohamed Salah,Liverpool,English Premier League,31,Right Winger,65.0,31
203,Michael Olise,Crystal Palace,English Premier League,22,Right Winger,50.0,22
861,Jonas Hofmann,Leverkusen,German Bundesliga,31,Attacking Midfield,13.0,31
872,Thomas Müller,Bayern Munich,German Bundesliga,34,Second Striker,10.0,34


In [147]:
df_fin = find_similar_players(eafc_pl_df, player_name = 'Kevin De Bruyne')
df_fin[['Player', 'Club', 'League', 'Age', 'Position', 'Value', 'Age']]

,Player,Club,League,Age,Position,Value,Age
442,İlkay Gündoğan,Barcelona,Spanish La Liga,33,Central Midfield,16.0,33
1020,Hakan Çalhanoğlu,Internazionale,Italian Serie A,30,Defensive Midfield,40.0,30
888,Joshua Kimmich,Bayern Munich,German Bundesliga,29,Right-Back,60.0,29
429,Toni Kroos,Real Madrid,Spanish La Liga,34,Central Midfield,12.0,34
882,Jonas Hofmann,Leverkusen,German Bundesliga,31,Attacking Midfield,13.0,31
